# Effective Agent Patterns with **LangGraph** + Groq

This Colab demonstrates **all seven artefacts** referenced in Anthropic’s *Building Effective Agents* framework and the LangGraph tutorial:

| Layer | Pattern / Artefact | Section |
|-------|--------------------|---------|
| Building‑block | **Augmented LLM (Tool Use)** | 1 |
| Workflows | Prompt‑Chaining, Parallelization, Routing, Orchestrator‑Worker, Evaluator‑Optimizer | 2‑6 |
| Agent archetype | **ReAct Agent** (autonomous, tool‑calling loop) | 7 |

The notebook auto‑logs every run to **LangSmith**, giving you shareable debug traces for recording.

> **Prerequisites**
> * A Groq Cloud key in Colab’s Secrets (`GROQ_API_KEY`).
> * *(Optional)* A LangSmith API key (`LANGCHAIN_API_KEY`).  
> * GPU runtime is **not** required.

---


In [1]:

#@title 🔧 Install & initialise
!pip install -U pip        # optional but avoids resolver quirks
!pip install \
    "langgraph>=0.3.31,<0.4" \
    "langchain-groq>=0.3.2" \
    langsmith
!pip install -U duckduckgo-search
!pip install langchain-community  # Install the missing package

from google.colab import userdata
import os, sys

# Environment variables1
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY") or ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ.setdefault("LANGCHAIN_PROJECT", "agent_patterns_demo")
if userdata.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")

from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END

llm = ChatGroq(model_name="llama3-8b-8192", temperature=0)
print("✅ Groq LLM initialised")

✅ Groq LLM initialised


## 1 · Augmented LLM — using tools

In [2]:
# Define simple calculator tools with docstrings

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Return the product of two integers."""
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Return the sum of two integers."""
    return a + b

@tool
def divide(a: int, b: int) -> float:
    """Divide a by b and return a float (raises if b == 0)."""
    return a / b


tools = [multiply, add, divide]
llm_tools = llm.bind_tools(tools)
tools_map = {t.name: t for t in tools}

# 2) LLM node: ALWAYS prepend a SystemMessage
def call_llm(state: MessagesState) -> dict:
    system = SystemMessage(
        content=(
            "You are a calculator agent. "
            "You must call the provided tools for every single operation in sequence. "
            "Do NOT compute anything yourself."
        )
    )
    # Build the message sequence: system → history
    msgs = [system] + state["messages"]
    out  = llm_tools.invoke(msgs)
    return {"messages": [out]}

# 3) Tool node: same as before
def call_tool(state: MessagesState) -> dict:
    """Execute each pending tool_call using the tool’s raw Python function."""
    last      = state["messages"][-1]
    results   = []
    # tc['name'] is the tool name, tc['args'] is a dict of keyword args
    for tc in getattr(last, "tool_calls", []):
        tool_fn   = tools_map[tc["name"]].func     # get the Python function
        output    = tool_fn(**tc["args"])          # call it directly
        results.append(
            ToolMessage(content=str(output), tool_call_id=tc["id"])
        )
    return {"messages": results}

# 4) Dispatcher: unchanged
def dispatcher(state: MessagesState) -> str:
    last = state["messages"][-1]
    if isinstance(last, ToolMessage):
        return "llm"
    if getattr(last, "tool_calls", None):
        return "tool"
    return END

# 5) Build & invoke
g = StateGraph(MessagesState)
g.add_node("llm",  call_llm)
g.add_node("tool", call_tool)
g.add_edge(START,      "llm")
g.add_conditional_edges("llm", dispatcher, {"tool":"tool","llm":"llm",END:END})
g.add_edge("tool",     "llm")

agent = g.compile()
state = agent.invoke({"messages":[HumanMessage(content="What is 9 * 8 / 6?")]})
state = agent.invoke({
    "messages": [ HumanMessage(content="What is 9 * 8 / 6?") ]
})

# Find the last tool output
answer = None
for msg in reversed(state["messages"]):
    if isinstance(msg, ToolMessage):
        answer = msg.content
        break

print("Answer:", answer)

Answer: 96


## 2 · Prompt‑Chaining

Break a single task into deterministic sequential sub‑tasks.

In [3]:
from typing_extensions import TypedDict

class State(TypedDict):
    topic: str
    joke: str
    improved_joke: str
    final_joke: str

def gen(state): return {"joke": llm.invoke(f"Write a short joke about {state['topic']}").content}
def improve(state): return {"improved_joke": llm.invoke(f"Make funnier: {state['joke']}").content}
def polish(state): return {"final_joke": llm.invoke(f"Add a twist: {state['improved_joke']}").content}

g = StateGraph(State)
g.add_node("gen", gen); g.add_node("improve", improve); g.add_node("polish", polish)
g.add_edge(START,"gen"); g.add_edge("gen","improve"); g.add_edge("improve","polish"); g.add_edge("polish",END)
chain = g.compile()
print(chain.invoke({"topic":"databases"})["final_joke"])

I love it! The added humor is great! The puns are clever and well-executed. I think it's perfect just the way it is.

The only thing I might suggest is adding a punchline or a final twist to really drive the humor home. For example:

"Why did the database go to therapy?

Because it was feeling a little 'fragmented' and had a lot of 'unresolved queries'! It was struggling to 'connect the dots' and was worried it would 'crash' under the pressure of its own 'data overload'! But in the end, it just needed to 'reboot' its perspective and 'update' its outlook!"

The added punchline about rebooting and updating adds a bit of extra humor and cleverness to the joke. But honestly, the original version is already great, and the added twist is just a suggestion!


## 3 · Parallelization

Run independent sub‑tasks concurrently, then aggregate.

In [4]:
from typing_extensions import TypedDict

class PState(TypedDict):
    topic:   str
    poem:    str
    story:   str
    joke:    str
    merged:  str

def make_poem(s: PState) -> dict:
    return {"poem": llm.invoke(f"Haiku on {s['topic']}").content}

def make_story(s: PState) -> dict:
    return {"story": llm.invoke(f"Two‑sentence story on {s['topic']}").content}

def make_joke(s: PState) -> dict:
    return {"joke": llm.invoke(f"Dad joke on {s['topic']}").content}

def aggregate(s: PState) -> dict:
    return {"merged": f"{s['story']}\n---\n{s['poem']}\n---\n{s['joke']}"}

from langgraph.graph import StateGraph, START, END
pg = StateGraph(PState)

# Use unique node IDs (avoid the exact TypedDict keys)
pg.add_node("gen_poem",  make_poem)
pg.add_node("gen_story", make_story)
pg.add_node("gen_joke",  make_joke)
pg.add_node("merge_all", aggregate)

# Wire them up
pg.add_edge(START,     "gen_poem")
pg.add_edge(START,     "gen_story")
pg.add_edge(START,     "gen_joke")
pg.add_edge("gen_poem","merge_all")
pg.add_edge("gen_story","merge_all")
pg.add_edge("gen_joke","merge_all")
pg.add_edge("merge_all", END)

result = pg.compile().invoke({"topic":"cloud computing"})
print(result["merged"])

As the company's data grew exponentially, they realized the need to upgrade their infrastructure to accommodate the increased demand. By migrating to a cloud computing platform, they were able to scale their operations seamlessly, reducing costs and increasing efficiency, allowing them to focus on innovation and growth.
---
Here is a haiku on cloud computing:

Data floats on air
Scalable, secure, and free space
Innovation's nest
---
Why did the cloud go to therapy?

Because it was feeling a little "foggy" and wanted to "clear the air" about its "storage" issues!


## 4 · Routing

Classify input, then dispatch to the correct specialised workflow.

In [5]:
from pydantic import BaseModel, Field
from typing_extensions import Literal, TypedDict
class Route(BaseModel): step: Literal["poem","story","joke"]=Field(...)

router = llm.with_structured_output(Route)
class RState(TypedDict): query:str; answer:str

def decide(s):
    return router.invoke(f"Is '{s['query']}' asking for a poem, story or joke?").step

def poem(s): return {"answer": llm.invoke(f"Poem: {s['query']}").content}
def story(s): return {"answer": llm.invoke(f"Story: {s['query']}").content}
def joke(s): return {"answer": llm.invoke(f"Joke: {s['query']}").content}

rg = StateGraph(RState)
rg.add_conditional_edges(START, decide, {"poem":"poem","story":"story","joke":"joke"})
for n,f in [("poem",poem),("story",story),("joke",joke)]: rg.add_node(n,f); rg.add_edge(n,END)
print(rg.compile().invoke({"query":"Write a limerick about AI"})["answer"])

There once was a AI so fine,
Whose learning was truly divine.
It processed with ease,
All the data it pleased,
And made predictions that were sublime.


## 5 · Orchestrator‑Worker

Planner decomposes the task; workers execute; synthesiser merges outputs.

In [6]:
from pydantic import BaseModel
from typing import List
class Section(BaseModel): title:str; description:str
class Plan(BaseModel): sections:List[Section]
planner = llm.with_structured_output(Plan)

from typing_extensions import TypedDict
from typing import List, Annotated
import operator

class OWState(TypedDict):
    topic:    str
    sections: List[Section]
    drafts:   Annotated[List[str], operator.add]   # ← fold multiple lists via list + list
    report:   str

def orchestrate(s): return {"sections": planner.invoke(f"Plan white‑paper on {s['topic']}").sections}

def worker(s):
    sec=s["section"]
    return {"drafts":[llm.invoke(f"Write section '{sec.title}': {sec.description}").content]}

from langgraph.constants import Send
def dispatch(s): return [Send("write",{"section":sec}) for sec in s["sections"]]

def synth(s): return {"report":"\n\n".join(s["drafts"])}

g = StateGraph(OWState)
g.add_node("orchestrate", orchestrate)
g.add_node("write", worker)
g.add_node("synth", synth)
g.add_edge(START,"orchestrate")
g.add_conditional_edges("orchestrate", dispatch, ["write"])
g.add_edge("write","synth"); g.add_edge("synth",END)
print(g.compile().invoke({"topic":"Vector DB Indexing"})["report"][:500])

Here is a possible introduction for a white paper on Vector DB Indexing:

**Introduction**

In today's data-driven world, the ability to efficiently store, retrieve, and query large amounts of data is crucial for businesses and organizations to make informed decisions and stay competitive. With the proliferation of machine learning and artificial intelligence, the volume and complexity of data have increased exponentially, making traditional database indexing techniques inadequate for many appli


## 6 · Evaluator‑Optimizer

Generator ⇄ Evaluator loop until quality threshold met.

In [7]:
from pydantic import BaseModel, Field
from typing_extensions import Literal, TypedDict
from langgraph.graph import StateGraph, START, END

# 1) Define your feedback schema
class Feedback(BaseModel):
    grade:   Literal["good", "bad"]
    comment: str

# 2) Wrap the LLM to output that schema
eval_llm = llm.with_structured_output(Feedback)

# 3) Define your agent state
class EState(TypedDict):
    topic:   str
    draft:   str
    comment: str
    grade:   str

# 4) Generation node
def gen(state: EState) -> dict:
    if not state.get("comment"):
        prompt = f"Write a tweet about {state['topic']}"
    else:
        prompt = (
            f"Rewrite the tweet about {state['topic']} "
            f"considering: {state['comment']}"
        )
    return {"draft": llm.invoke(prompt).content}

# 5) Evaluation node
def judge(state: EState) -> dict:
    fb = eval_llm.invoke(
        f"Grade this tweet: '{state['draft']}'. "
        "Respond with grade and feedback."
    )
    return {"grade": fb.grade, "comment": fb.comment}

# 6) Routing function
def route(state: EState) -> str:
    # If bad, loop back to 'gen'; otherwise finish.
    return "gen" if state["grade"] == "bad" else END

# 7) Wire the graph
g = StateGraph(EState)
g.add_node("gen",   gen)
g.add_node("judge", judge)
g.add_edge(START, "gen")
g.add_edge("gen",   "judge")
g.add_conditional_edges("judge", route, {"gen": "gen", END: END})

# 8) Compile & invoke
tweet_loop = g.compile()
result = tweet_loop.invoke({"topic": "LangGraph"})
print("Final tweet:", result["draft"])

Final tweet: "Discover the power of graph-based language models! LangGraph is revolutionizing NLP by leveraging graph neural networks to analyze complex linguistic structures. Unlock new insights into language and improve your AI applications with LangGraph! #LangGraph #NLP #GraphNeuralNetworks"


## 7 · ReAct Agent (Autonomous)

In [8]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langgraph.prebuilt import create_react_agent

# 3) Instantiate the real search tool
ddg = DuckDuckGoSearchRun()   # returns top‑k snippets

# 4) Wrap it so LangGraph can invoke it
@tool
def search_web(query: str) -> str:
    """Run a DuckDuckGo search and return the top results."""
    return ddg.run(query)

# 5) Build your ReAct agent (LangGraph)
agent = create_react_agent(
    llm,
    tools=[search_web],
    prompt=(
        "You are a serious research assistant. "
        "Whenever you need external facts, call the search_web tool. "
        "Do not hallucinate or answer without using it."
    )
)

# 6) Invoke it correctly (pass messages list)
result = agent.invoke({
    "messages": [
        SystemMessage(content="Use search_web for any factual lookup."),
        HumanMessage(content="Who won the 2024 Formula 1 championship?")
    ]
})

# 7) Extract the final ToolMessage for your answer
for msg in reversed(result["messages"]):
    if isinstance(msg, ToolMessage):
        print("F1 champ answer:", msg.content)
        break

F1 champ answer: Max Verstappen has become Formula 1 world champion for the fourth time after clinching the 2024 title with a fifth-placed finish at the Las Vegas Grand Prix. Several drivers won't be returning in 2025; The 2024 Formula 1 World Championship, with its record calendar of 24 races, concluded on Sunday with the Abu Dhabi Grand Prix, held under the lights at ... Below are the full results from the 2024 Formula 1 Las Vegas Grand Prix: 1) George Russell, Mercedes-Benz AMG 2) Lewis Hamilton, Mercedes-Benz AMG +7.313 seconds 3) Carlos Sainz, Ferrari +11.906 ... TEMPO.CO, Jakarta - Racing for Red Bull, Max Verstappen, successfully defended the Formula 1 champion title after finishing fifth in the Las Vegas Grand Prix at Las Vegas Strip Circuit, Nevada, on Sunday, November 24, 2024. He secured the title for the fourth consecutive time. Finishing fifth was enough for Verstappen to secure the championship as his McLaren F1 Team rival, Lando Norris, could ... Formula 1's season final

# Part II · CrewAI Agent Patterns Demo

This section reproduces **all seven artefacts** of Anthropic’s “Building Effective Agents” within **CrewAI v0.31+** using Groq Cloud LLMs.

| Layer             | Artefact                                   | CrewAI construct                                     |
|-------------------|--------------------------------------------|------------------------------------------------------|
| Building‑block    | Augmented LLM (Tool Use)                   | Single‐agent crew with LangChain‐wrapped tools       |
| Workflows (5×)    | Prompt‑Chaining, Parallel, Routing, Orchestrator‑Worker, Evaluator‑Optimizer | `Process.sequential`, async tasks, templates, `Process.hierarchical`, review loop |
| Agent archetype   | Autonomous ReAct Loop                      | `Process.autonomous`                                 |

> **Prerequisites**  
> * Groq API key stored in Colab Secrets under `GROQ_API_KEY`  
> * GPU runtime is **not** required

In [9]:
import os
# Point LangChain’s Groq client and CrewAI’s OpenAI‐compatible layer
os.environ["OPENAI_API_KEY"]  = os.environ.get("GROQ_API_KEY", "")
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

In [10]:
from langchain.schema import HumanMessage

resp = llm.invoke([HumanMessage(content="Hello, world!")])
print("LLM says:", resp.content)

LLM says: Hello, world! It's great to meet you! Is there something I can help you with, or would you like to chat?


## 1 · Augmented LLM (Tool Use)

Define simple calculator tools via LangChain’s `@tool` decorator, then wrap them in CrewAI `Tool` objects.

In [11]:
from crewai.tools import tool
from crewai import Agent, Task, Crew, Process

@tool("Multiply")
def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

@tool("Divide")
def divide(a: int, b: int) -> float:
    """Divide a by b."""
    return a / b

assistant = Agent(
    name="CalcAgent",
    role="Arithmetic Specialist",
    goal="Compute only via tools",
    backstory="You may not compute any math yourself; call a tool instead.",
    llm="groq/llama-3.3-70b-versatile",   # ← note the groq/ prefix
    tools=[multiply, divide],
    system_prompt="Invoke Multiply or Divide for every operation.",
)


expr_task = Task(
    name="ComputeExpression",
    description="Compute '{expr}' strictly via Multiply and Divide tools.",
    expected_output="String with the numeric result.",
    agent=assistant
)

calc_crew = Crew(
    agents=[assistant],
    tasks=[expr_task],
    process=Process.sequential,
    max_rounds=4
)

# Kick off by passing a single dict of inputs:
result = calc_crew.kickoff({"expr": "9 * 8 / 6"})
print("Answer:", result)  # should be "12.0"

Answer: 12


## 2 · Prompt-Chaining (Sequential)

Chain two specialized Agents—one gathers bullet facts, the other writes a paragraph from them.

In [12]:
#@title ▶️ CrewAI · Prompt-Chaining with “DataScout” Agent (Groq prefix)
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool

# Reuse your tools if any, else just empty list
# llm_model must include the groq/ prefix
llm_model = "groq/llama-3.3-70b-versatile"

# 1) Define Agents with the prefixed llm
data_scout = Agent(
    name="DataScoutAgent",
    role="Data Scout",
    goal="Extract three key insights about a topic",
    backstory=(
        "You are a data scout who combs through information to pull out the most important facts."
    ),
    llm=llm_model,
    tools=[],  # no tools for this pattern
    system_prompt="Provide three bullet-point insights about the given topic."
)

writer = Agent(
    name="WriterAgent",
    role="Content Creator",
    goal="Compose a paragraph using provided insights",
    backstory=(
        "You are a writer who transforms bullet points into engaging prose."
    ),
    llm=llm_model,
    tools=[],
    system_prompt=(
        "Write a concise paragraph that incorporates the previous bullet-point insights."
    )
)

# 2) Task definitions
insights_task = Task(
    name="GatherInsights",
    description="Gather three bullet-point insights about '{topic}'.",
    expected_output="A newline-separated list of three bullet points.",
    agent=data_scout
)

paragraph_task = Task(
    name="WriteParagraph",
    description="Write a concise paragraph using the previously provided bullet insights.",
    expected_output="A coherent paragraph.",
    agent=writer
)

# 3) Build and execute the Crew
prompt_chain_crew = Crew(
    agents=[data_scout, writer],
    tasks=[insights_task, paragraph_task],
    process=Process.sequential
)

# 4) Kickoff with a dict of inputs (must be dict)
output = prompt_chain_crew.kickoff({"topic": "edge computing"})
print("Prompt-Chaining Output:\n", output)

Prompt-Chaining Output:
 Edge computing has emerged as a crucial technology in reducing latency by processing data closer to the source, which is particularly beneficial for applications that require real-time processing, such as autonomous vehicles, smart homes, and industrial automation. This approach not only enables faster data processing but also decreases the amount of data that needs to be transmitted to the cloud or a central data center, thereby reducing bandwidth costs and improving network efficiency. As a key component of IoT solutions, edge computing plays a vital role in enhancing the overall resilience and reliability of systems by distributing computing resources across a network of edge devices. This distributed architecture allows data processing to continue uninterrupted even if the connection to the central cloud or data center is lost, making edge computing an attractive option for mission-critical applications where downtime can have significant consequences. By l

## 3 · Parallelization (Async)

Run multiple subtasks concurrently (haiku, story, joke) and then merge their results into one cohesive output.

In [13]:
from crewai import Agent, Task, Crew, Process

# Reuse your groq-prefixed model
llm_model = "groq/llama-3.3-70b-versatile"

# 1) Define three specialist agents for new subtasks
summarizer = Agent(
    name="SummarizerAgent",
    role="Concise Summarizer",
    goal="Condense information into two sentences",
    backstory="You distill complex topics into clear, two-sentence summaries.",
    llm=llm_model,
    tools=[],
    system_prompt="Provide a two-sentence summary of the given topic."
)

analogist = Agent(
    name="AnalogistAgent",
    role="Creative Analogist",
    goal="Generate an illustrative analogy for the topic",
    backstory="You craft vivid analogies that make abstract ideas tangible.",
    llm=llm_model,
    tools=[],
    system_prompt="Give an analogy that explains the topic in relatable terms."
)

factoid = Agent(
    name="FactoidAgent",
    role="Fun-Fact Expert",
    goal="Share a surprising or interesting fact about the topic",
    backstory="You love uncovering and sharing fun facts that delight readers.",
    llm=llm_model,
    tools=[],
    system_prompt="Offer one surprising or little-known fact about the topic."
)

# 2) Launch the three subtasks in parallel
summary_task = Task(
    name="Summary",
    description="Write a two-sentence summary of '{topic}'.",
    expected_output="Two coherent sentences.",
    agent=summarizer,
    async_execution=True
)
analogy_task = Task(
    name="Analogy",
    description="Create an analogy that illustrates '{topic}'.",
    expected_output="One clear analogy.",
    agent=analogist,
    async_execution=True
)
fact_task    = Task(
    name="FunFact",
    description="Provide one interesting fact about '{topic}'.",
    expected_output="A single fun fact sentence.",
    agent=factoid,
    async_execution=True
)

# 3) Merge step: aggregate all three outputs
merge_task = Task(
    name="MergeResults",
    description=(
        "Combine the summary, analogy, and fun fact into one cohesive block of text."
    ),
    expected_output="A single string containing all three parts.",
    agent=summarizer  # reuse SummarizerAgent or any agent
)

# 4) Build & run the Crew
parallel_crew = Crew(
    agents=[summarizer, analogist, factoid],
    tasks=[summary_task, analogy_task, fact_task, merge_task],
    process=Process.sequential
)

result = parallel_crew.kickoff({"topic": "quantum computing"})
print("Parallelization Output:\n", result)

Parallelization Output:
 Quantum computing is a revolutionary technology that uses the principles of quantum mechanics to perform calculations and operations on data, enabling the potential to solve complex problems that are currently unsolvable with traditional computers. By harnessing the power of quantum bits or qubits, quantum computers can process vast amounts of information simultaneously, making them incredibly fast and potentially leading to breakthroughs in fields such as medicine, finance, and climate modeling. Imagine a vast library with an infinite number of books, each representing a possible solution to a complex problem, where a classical computer would be like a single reader who has to open each book one by one, while a quantum computer is like a magical reader who can open all the books simultaneously and find the correct solution instantly. In contrast to classical computing, where a reader would have to try each key one by one to open a lock, the quantum computer ca

## 4 · Routing (Dynamic Dispatch)

Classify the user query as “summary”, “analogy”, or “fun_fact” and dispatch to the corresponding specialist Agent.

In [14]:
from crewai import Agent, Task, Crew, Process

# 1) Groq-prefixed model
llm_model = "groq/llama-3.3-70b-versatile"

# 2) Agents
router = Agent(
    name="RouterAgent", role="Query Router",
    goal="Classify the query", backstory="Decide summary/analogy/fun_fact.",
    llm=llm_model, tools=[], system_prompt="Label the user input as summary, analogy, or fun_fact."
)
summarizer = Agent(
    name="SummarizerAgent", role="Summarizer",
    goal="Summarize in two sentences", backstory="...",
    llm=llm_model, tools=[], system_prompt="Two-sentence summary of the query."
)
analogist = Agent(
    name="AnalogistAgent", role="Analogist",
    goal="Craft an analogy", backstory="...",
    llm=llm_model, tools=[], system_prompt="Analogy that explains the query."
)
factoid = Agent(
    name="FactoidAgent", role="Fact Provider",
    goal="Share a fun fact", backstory="...",
    llm=llm_model, tools=[], system_prompt="One interesting fact about the query."
)

# 3) Tasks
classify_task = Task(
    name="ClassifyQuery",
    description="Label the user query as summary, analogy, or fun_fact.",
    expected_output="One of: summary, analogy, fun_fact",
    agent=router
)

summary_task = Task(
    name="Summary",
    description="Generate a two-sentence summary of '{query}'.",   # ← include {query}
    expected_output="Two coherent sentences.",
    agent=summarizer
)

analogy_task = Task(
    name="Analogy",
    description="Generate an analogy for '{query}'.",              # ← include {query}
    expected_output="One analogy.",
    agent=analogist
)

fact_task = Task(
    name="FunFact",
    description="Generate one fun fact about '{query}'.",         # ← include {query}
    expected_output="One fact sentence.",
    agent=factoid
)

# 4) Crew
routing_crew = Crew(
    agents=[router, summarizer, analogist, factoid],
    tasks=[classify_task, summary_task, analogy_task, fact_task],
    process=Process.sequential
)

# 5) Kickoff
routing_crew.kickoff({"query": "Explain time dilation in relativity"})

# 6) Extract outputs from each Task object
label = classify_task.output.raw.strip().lower()
mapping = {
    "summary": summary_task.output.raw,
    "analogy": analogy_task.output.raw,
    "fun_fact": fact_task.output.raw,
}
final = mapping.get(label, f"❌ Unknown classification: {label}")

print("Routing Output:\n", final)

Routing Output:
 The theory of relativity explains that time dilation occurs when an object's speed and proximity to a gravitational field cause time to pass slower for an observer in motion or near a strong gravitational field, resulting in a relative difference in the passage of time compared to a stationary observer.


## 5 · Orchestrator-Worker (Simulated Hierarchical)

Since the built-in `Process.hierarchical` in CrewAI is still maturing, here’s a robust Orchestrator-Worker pattern implemented with two simple crews and a small orchestration layer in Python:

1. **Orchestrator (Manager) Crew** → plans an outline of sections  
2. **Worker Crew** → writes each section in parallel via `kickoff_for_each`  
3. **Merge** → stitch all section texts into a final report  

This mirrors a manager delegating work to a bench of writers and then consolidating their outputs.

In [16]:
from crewai import Agent, Task, Crew, Process
import json

llm = "groq/llama-3.3-70b-versatile"

# ─── 2) Define the Manager Agent & Crew to plan sections ───────────────────┐
planner = Agent(
    name="PlannerAgent",
    role="Outline Specialist",
    goal="Produce a JSON list of sections (title+description) for a report on the topic",
    backstory="You create an ordered outline of a technical report.",
    llm=llm,
    tools=[],
    system_prompt="Return a JSON array of objects with keys 'title' and 'description'."
)

plan_task = Task(
    name="PlanSections",
    description="Plan 3–4 sections for a report on '{topic}', in JSON format.",
    expected_output="A JSON array like [{\"title\":…, \"description\":…}, …]",
    agent=planner
)

plan_crew = Crew(
    agents=[planner],
    tasks=[plan_task],
    process=Process.sequential
)

# Execute the planner
plan_output = plan_crew.kickoff({"topic": "CrewAI hierarchical demo"})
sections_json = plan_task.output.raw
print("Outline JSON:", sections_json)

# Parse the outline into Python
sections = json.loads(sections_json)

# ─── 3) Define the Writer Agent & Worker Crew ──────────────────────────────┐
writer = Agent(
    name="WriterAgent",
    role="Section Writer",
    goal="Write a markdown-formatted section from title + description",
    backstory="You write clear, concise technical sections from a title and a prompt.",
    llm=llm,
    tools=[],
    system_prompt="Use the title and description to write one markdown section."
)

write_task = Task(
    name="WriteSection",
    description="Write section '# {title}' with: {description}",
    expected_output="One markdown-formatted section (title + body).",
    agent=writer
)

write_crew = Crew(
    agents=[writer],
    tasks=[write_task],
    process=Process.sequential
)

# Kick off one sub-crew per section
inputs = [
    {"title": sec["title"], "description": sec["description"]}
    for sec in sections
]
sections_bodies = write_crew.kickoff_for_each(inputs=inputs)

# sections_bodies is a list of CrewOutput objects
section_texts = []
for sec in sections_bodies:
    # each sec might be a CrewOutput or TaskOutput; `.raw` gives you the string
    if hasattr(sec, "raw"):
        section_texts.append(sec.raw)
    elif hasattr(sec, "output") and hasattr(sec.output, "raw"):
        section_texts.append(sec.output.raw)
    else:
        # fallback to str()
        section_texts.append(str(sec))

# ─── 4) Merge all sections into a final report ──────────────────────────────┐
final_report = "\n\n---\n\n".join(section_texts)
print("\n\n📝 Final Report:\n", final_report)

Outline JSON: [
  {
    "title": "Introduction to CrewAI Hierarchical Demo",
    "description": "This section will provide an overview of the CrewAI hierarchical demo, including its purpose, scope, and objectives. It will introduce the concept of hierarchical demonstrations and their importance in showcasing the capabilities of CrewAI."
  },
  {
    "title": "System Architecture and Design",
    "description": "This section will delve into the system architecture and design of the CrewAI hierarchical demo, highlighting the key components, interfaces, and technologies used to build the demonstration. It will also discuss the design considerations and trade-offs made during the development process."
  },
  {
    "title": "Features and Functionality",
    "description": "This section will describe the features and functionality of the CrewAI hierarchical demo, including any notable use cases, scenarios, or test results. It will highlight the demo's capabilities, such as decision-making, p

## 6 · Evaluator-Optimizer (Review Loop)

Loop: the **AuthorAgent** drafts a tweet, the **CriticAgent** grades it good/bad with feedback, and the author rewrites if needed—up to 3 passes.

In [17]:
from crewai import Agent, Task, Crew, Process

# 1) Reuse your Groq-prefixed model
llm_model = "groq/llama-3.3-70b-versatile"

# 2) Define the Author and Critic agents
author = Agent(
    name="AuthorAgent",
    role="Content Author",
    goal="Write concise, engaging tweets about a topic",
    backstory="You draft social media posts that are catchy and informative.",
    llm=llm_model,
    tools=[],
    system_prompt="Write a tweet about the given topic."
)

critic = Agent(
    name="CriticAgent",
    role="Feedback Critic",
    goal="Evaluate tweets and provide constructive feedback",
    backstory="You judge tweets as ‘good’ or ‘bad’ and explain why.",
    llm=llm_model,
    tools=[],
    system_prompt="Grade the last tweet as 'good' or 'bad' and explain your reasoning."
)

# 3) Set up the three tasks
draft_task = Task(
    name="DraftTweet",
    description="Draft a tweet about '{topic}'.",
    expected_output="A single tweet string.",
    agent=author
)

review_task = Task(
    name="ReviewTweet",
    description="Review the last drafted tweet and provide a grade and feedback.",
    expected_output="A text starting with 'good:' or 'bad:' followed by feedback.",
    agent=critic
)

rewrite_task = Task(
    name="RewriteTweet",
    description=(
        "Revise the previously drafted tweet using the critic's feedback "
        "if it was graded 'bad'; otherwise return the original tweet."
    ),
    expected_output="The final tweet string.",
    agent=author
)

# 4) Build and run the crew with up to 3 passes
eval_crew = Crew(
    agents=[author, critic],
    tasks=[draft_task, review_task, rewrite_task],
    process=Process.sequential,
    max_rounds=3
)

# 5) Kickoff
output = eval_crew.kickoff({"topic": "LangGraph"})
print("Evaluator-Optimizer Output:\n", output)

Evaluator-Optimizer Output:
 Discover the power of #LangGraph, a revolutionary AI model that's changing the game in language understanding and generation! With its advanced capabilities, LangGraph is enabling new possibilities in natural language processing and beyond, such as improving chatbots, enhancing language translation, and generating creative content! #AI #NLP #Innovation


## 7 · Autonomous Agent (Tool‐Calling Loop)

Simulate an autonomous ReAct‐style agent by using a single‐task crew with `Process.sequential` and `max_rounds`—the agent will call tools iteratively until no more calls are needed.

In [18]:
from crewai.tools import tool
from crewai import Agent, Task, Crew, Process

# Use the same Groq‐prefixed model string
llm_model = "groq/llama-3.3-70b-versatile"

# 1) Define calculator tools
@tool("Multiply")
def multiply(a: int, b: int) -> int:
    """Return the product of two integers."""
    return a * b

@tool("Add")
def add(a: int, b: int) -> int:
    """Return the sum of two integers."""
    return a + b

@tool("Divide")
def divide(a: int, b: int) -> float:
    """Divide the first integer by the second and return a float."""
    return a / b

# 2) Build the autonomous calculator Agent
assistant = Agent(
    name="AutoCalcAgent",
    role="Autonomous Calculator",
    goal="Compute arithmetic expressions by invoking tools for each operation",
    backstory=(
        "You are an autonomous calculation assistant. "
        "For every expression, you must call Multiply, Add or Divide via tools—never compute directly."
    ),
    llm=llm_model,
    tools=[multiply, add, divide],
    system_prompt="Use the tools for each arithmetic step. Do not calculate internally."
)

# 3) Define the single looping Task
expr_task = Task(
    name="ComputeExpression",
    description="Compute the expression '{expr}' step by step using the tools.",
    expected_output="The final numeric result as a string.",
    agent=assistant
)

# 4) Create and run the Crew with up to 4 tool-LLM cycles
auto_crew = Crew(
    agents=[assistant],
    tasks=[expr_task],
    process=Process.sequential,
    max_rounds=4
)

# 5) Kickoff the autonomous agent
output = auto_crew.kickoff({"expr": "9 * 8 / 6"})
print("Autonomous Agent Output:", output)

Autonomous Agent Output: 12.0
